# This progream creates concordia index and run information needs queries
# NOTE: Simply run the dependencies block and then load stored indeces instead of constructing one (skip code blocks up until the loading section)

In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re,os
import nltk
import pickle 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
ps = PorterStemmer()
import numpy as np
from operator import itemgetter

# Set project directory
#os.chdir()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
os.listdir()

['.ipynb_checkpoints',
 'A3-Q1-2-3-final.ipynb',
 'desktop.ini',
 'documents-ai',
 'documents-ai-2',
 'documents-ai-3',
 'documents-ai-filtered',
 'documents-final',
 'Filtering-duplicates.ipynb',
 'final_index_concordia',
 'Preparing-documents.ipynb',
 'Q4-AI-index.ipynb',
 'term_freq_concordia',
 'urls_run_final.csv']

In [ ]:
"""class custom_scrape():
    
    docID_link_map = {}
    #my_url2 = "https://www.concordia.ca/research/news.html"
    omit = "/news/stories"
    doc_counter = 1

    def ret_filtered_links(self,url,omit):

        link_temp = urlopen(url)
        page_html = link_temp.read()
        link_temp.close()
        page_soup = soup(page_html,"html.parser")
        research_page = page_soup.findAll('a',href=True)

        links = []
        filtered_links = []

        for i,link in enumerate(research_page):
                links.append(link["href"])

        for link in links:
            if "/stories" in link:
                if link!=custom_scrape.omit:
                    filtered_links.append(link)

        for i in range(len(filtered_links)):
            filtered_links[i] = "https://concordia.ca"+filtered_links[i]
            
        filtered_links_final = list(set(filtered_links))

        return filtered_links_final

    def process_filtered_links(self,filtered_links):
        for i,link in enumerate(filtered_links):
            link_temp = urlopen(link)
            page_html = link_temp.read()
            link_temp.close()
            page_soup = soup(page_html,"html.parser")
            txt_doc = page_soup.get_text()
            os.chdir("documents")
            text_file = open(str(custom_scrape.doc_counter), "w")
            text_file.write(txt_doc)
            text_file.close()
            os.chdir("..")
            custom_scrape.docID_link_map[custom_scrape.doc_counter]=link
            custom_scrape.doc_counter+=1
            
            if custom_scrape.doc_counter%5==0:
                print(str(custom_scrape.doc_counter) + "-doc has been processed!.")

       
    
    def scrape_docs(self,url,limit = 3):
        
        for i in range(limit):
            if i == 0:
                filtered_links = self.ret_filtered_links(url,custom_scrape.omit)
                self.process_filtered_links(filtered_links)
            else:
                new_url = url+"?page="+str(i+1)
                filtered_links = self.ret_filtered_links(new_url,custom_scrape.omit)
                self.process_filtered_links(filtered_links)
                
    def scrape_root(self,my_url):
        
        link = urlopen(my_url)
        page_html = link.read()
        link.close()
        page_soup = soup(page_html,"html.parser")
        research_page = page_soup.findAll('a',href=True)

        soc_media = ["twitter","linkedin","facebook","instagram"]
        links = []
        filtered_links = []
        for i,link in enumerate(research_page):
                links.append(link["href"])

        for link in links:
            if "html" in link:
                filtered_links.append(link)

        for media in soc_media:
            for link in filtered_links:
                if media in link:
                    filtered_links.remove(link)

        for i in range(len(filtered_links)):
            filtered_links[i] = "https://concordia.ca"+filtered_links[i]

        for i,link in enumerate(filtered_links):

            link_temp = urlopen(link)
            page_html = link_temp.read()
            link_temp.close()
            page_soup = soup(page_html,"html.parser")
            txt_doc = page_soup.get_text()
            os.chdir("documents")
            text_file = open(str(i+1), "w")
            text_file.write(txt_doc)
            text_file.close()
            os.chdir("..")
            custom_scrape.docID_link_map[custom_scrape.doc_counter]=link
            custom_scrape.doc_counter+=1

"""
            



# Index construction and compression

In [3]:
"SPIMI Algorithm class"

class Spimi():
    
    "Block counter that will be incremented as the memory gets full"
    b_counter = 1 
    "All the tags to be filtered"
    TAGS = "<TITLE>,</TITLE>,<TEXT>,</TEXT>,<REUTERS>,</REUTERS>,<DATE>,</DATE>,<TOPICS>,</TOPICS>,<PLACES>,</PLACE>,<PEOPLE>,</PEOPLE>,<COMPANIES>,</COMPANIES>,<UNKNOWN>,</UNKNOWN>,<DATELINE>,</DATELINE>,<BODY>,</BODY>,<AUTHOR>,</AUTHOR>,<ORGS>,</ORGS>,<EXCHANGES>,</EXCHANGES>,<D>,</D>"
    TAGS = TAGS.split(",")
    Dictionary={}
    term_freq = {}
    
    
    """Function: Constructor with two main variables, document ID, and 
       a last_position variable that stores the position of the pointer
       in the file being read.
    """
    def __init__(self):
        self.docID = 1
        self.last_pos = 0
        
    "Function: sorts a dictionary alphabetically"
    def sort_dict(self,D):
        sorted_dict = {}
        for key in sorted(D):
            sorted_dict[key]=D[key]
        return sorted_dict
    
    "Writes a block into disk."
    def write_block(self,outputFile,Dictionary):
        with open(outputFile, 'wb') as fp:
                        pickle.dump(Dictionary, fp, protocol=pickle.HIGHEST_PROTOCOL)
                
    "Filters the tags in the TAGS variable that show up in a read line from the file."            
    def filter_tags(self,line):
        for tag in Spimi.TAGS:
            line=re.sub(tag," ", line)
        return line
    
    """Function: This is a compression method that helps in removing a word if the number of
       identical consecutive characters is greater than two. For example: informmmation
    """           
    def repetitive_check(self,token):
        consq_chars = re.findall(r"((\w)\2{2,})",token)
        if len(consq_chars) > 0 :
            return True
        else:
            return False

    "Function: This method takes in a token, and updates the dictionary."
    def updateDict(self,Dictionary,token,stopw_150=False,stem=False):
                
            token = re.sub("[^a-zA-Z]+", " ", token)

            
            "If a token contains more than one word, it will be omitted. For example: 'america ru' "
            if len(token.split(" "))>1: 
                return
            
            "Omits a token that has three consecutive identical characters"
            if self.repetitive_check(token) == True:
                return
            
            "Checks for stopwords"           
            if stopw_150 == True:
                if  token in set(stopwords.words("english")[:150]):
                    return 
                
            "Stems the token"         
            if stem == True:
                token = ps.stem(token)
            
            "If the token doesn't already exist, add it"
            "omit tokens that have  length < 3"
            if token not in Dictionary.keys() or token =="ai"and len(token)>2 :          
                Dictionary[token] = []
                Dictionary[token].append(self.docID)
                Spimi.term_freq[token]=1
            else:
                "If the token exists, and the corresponding docID isn't already stored, append it to the tokens postings list."
                if len(token)<3 and token != "ai":
                    return
                #if self.docID not in Dictionary[token]:
                Dictionary[token].append(self.docID)
                Spimi.term_freq[token]+=1
                
            l = Dictionary[token]
            Dictionary[token] = list(set(l))

    "Function: Spimi algorithm"
    def SPIMI_INVERT(self,txt_file, case_folding=False, no_num=False,stopw_30=False,stopw_150=False,stem=False):
        
        postings_list = None
        output_file = "block"+str(self.b_counter) 
        file = open(txt_file,encoding="utf8",errors="ignore",mode="r")
        txt_file=txt_file.split(".")[0]
        self.docID = int(txt_file)
 
        for line in file:
            
            "Filter htmt/xml tags"    
            line = self.filter_tags(line) 
            "Filters non-alphanumeric characters"
            line = re.sub("\W+"," ", line) 
            "Casefold"
            line = line.lower()  
            
            for word in line.split():

                token = word
        
                "Update dictionary with each token."
                self.updateDict(Spimi.Dictionary,token,stopw_150=stopw_150,stem=stem)
                

        if self.docID%1000==0:

            print("MEMORY FULL - WRITING TO DISK. - Document:" + str(self.docID))

            Spimi.Dictionary = self.sort_dict(Spimi.Dictionary)
            
            Spimi.b_counter+=1 

            "Access disk folder"
            os.chdir("disk") 
            self.write_block(output_file,Spimi.Dictionary)
            "Return to reuters directory"
            os.chdir("..") 
        elif self.docID == 40207:
            "Access disk folder"
            os.chdir("disk") 
            self.write_block(output_file,Spimi.Dictionary)
            "Return to reuters directory"
            os.chdir("..") 
            



# Preparing documents to be fed into spimi


In [188]:
os.listdir()

['.ipynb_checkpoints',
 'A3-Q1-2-3-final.ipynb',
 'A3-Q1-2-3.ipynb',
 'desktop.ini',
 'documents',
 'documents-final',
 'documents2',
 'documents2-p2',
 'Final_Index',
 'Preparing-documents.ipynb',
 'term_freq',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'urls_run2.csv',
 'urls_run_final.csv']

In [25]:
os.chdir("..")

In [189]:
os.chdir("documents-final")
documents = os.listdir()
documents.remove("disk")
#documents.remove(".DS_Store")
documents = sorted(documents, key=lambda x: int(x.split('.')[0]))

#documents = sorted(documents)
os.chdir("..")
#documents

In [190]:
documents

['1.txt',
 '2.txt',
 '3.txt',
 '10.txt',
 '11.txt',
 '12.txt',
 '13.txt',
 '14.txt',
 '18.txt',
 '19.txt',
 '20.txt',
 '21.txt',
 '23.txt',
 '24.txt',
 '25.txt',
 '27.txt',
 '30.txt',
 '33.txt',
 '34.txt',
 '45.txt',
 '46.txt',
 '48.txt',
 '54.txt',
 '71.txt',
 '74.txt',
 '84.txt',
 '91.txt',
 '95.txt',
 '100.txt',
 '101.txt',
 '102.txt',
 '103.txt',
 '104.txt',
 '105.txt',
 '106.txt',
 '108.txt',
 '109.txt',
 '111.txt',
 '112.txt',
 '114.txt',
 '115.txt',
 '118.txt',
 '119.txt',
 '120.txt',
 '121.txt',
 '122.txt',
 '123.txt',
 '124.txt',
 '125.txt',
 '126.txt',
 '127.txt',
 '128.txt',
 '129.txt',
 '130.txt',
 '131.txt',
 '132.txt',
 '134.txt',
 '135.txt',
 '136.txt',
 '137.txt',
 '138.txt',
 '139.txt',
 '141.txt',
 '142.txt',
 '143.txt',
 '145.txt',
 '147.txt',
 '148.txt',
 '150.txt',
 '153.txt',
 '154.txt',
 '156.txt',
 '157.txt',
 '158.txt',
 '159.txt',
 '160.txt',
 '163.txt',
 '164.txt',
 '167.txt',
 '168.txt',
 '169.txt',
 '170.txt',
 '171.txt',
 '172.txt',
 '173.txt',
 '174.txt',

# Index construction

In [191]:
os.chdir("documents-final")
spimi_obj = Spimi()
for txt_file in documents:
     spimi_obj.SPIMI_INVERT(txt_file,stem=False,stopw_150=False)

        

MEMORY FULL - WRITING TO DISK. - Document:1000
MEMORY FULL - WRITING TO DISK. - Document:3000
MEMORY FULL - WRITING TO DISK. - Document:5000
MEMORY FULL - WRITING TO DISK. - Document:10000
MEMORY FULL - WRITING TO DISK. - Document:11000
MEMORY FULL - WRITING TO DISK. - Document:12000
MEMORY FULL - WRITING TO DISK. - Document:13000
MEMORY FULL - WRITING TO DISK. - Document:14000
MEMORY FULL - WRITING TO DISK. - Document:15000
MEMORY FULL - WRITING TO DISK. - Document:16000
MEMORY FULL - WRITING TO DISK. - Document:17000
MEMORY FULL - WRITING TO DISK. - Document:20000
MEMORY FULL - WRITING TO DISK. - Document:24000
MEMORY FULL - WRITING TO DISK. - Document:25000
MEMORY FULL - WRITING TO DISK. - Document:27000
MEMORY FULL - WRITING TO DISK. - Document:32000
MEMORY FULL - WRITING TO DISK. - Document:35000
MEMORY FULL - WRITING TO DISK. - Document:36000
MEMORY FULL - WRITING TO DISK. - Document:37000
MEMORY FULL - WRITING TO DISK. - Document:40000


In [192]:
"Access disk folder."
os.chdir("disk") 
Blocks={}
"Store all blocks in a dictionary called Blocks."
for i in range(len(os.listdir())):
    with open("block"+str(i+1), 'rb') as fp:
        block = pickle.load(fp)
        Blocks["block"+str(i+1)] = block

os.chdir("..")

Blocks.keys()

dict_keys(['block1', 'block2', 'block3', 'block4', 'block5', 'block6', 'block7', 'block8', 'block9', 'block10', 'block11', 'block12', 'block13', 'block14', 'block15', 'block16', 'block17', 'block18', 'block19', 'block20'])

# Merge blocks

In [ ]:
"Function: returns a list of the names of all the blocks that a term exists in."
def term_block_mapping(term, Blocks):
    term_block_mapping = []
    for block in Blocks:
        if term in Blocks[block].keys():
            term_block_mapping.append(block)
            
    return term_block_mapping


Merged_blocks = {}
for block in Blocks.keys():
    Merged_blocks.update(Blocks[block])
print("num of terms: " + str(len(Merged_blocks)))
all_terms = list(Merged_blocks.keys())


Merged_block = {}
Final_index = {}
MB_count=1
"""
For every term (in sorted list of terms), return all its postings list,
merge them, and then store a mapping of that term and its merged postings into a temporary
dictionary untill 25000 terms are reached. If 25000 terms are reached, store that temporary
dictionary into a final index dictionary, reset the temporary dictionary variable, and repeat
the process.
"""   
for term in sorted(all_terms):
        
    term_in_blocks = term_block_mapping(term,Blocks)

    Merged_block[term] = []

    for block in term_in_blocks:
        Merged_block[term].extend(Blocks[block][term])
    
    l = Merged_block[term]
    Merged_block[term] = list(set(l))
        
            



In [196]:
os.chdir("..")

In [204]:
# save term freq and final index
#s = Spimi()
#s.write_block("term_freq_concordia",Spimi.term_freq)
#s.write_block("final_index_concordia",Merged_block)

# Load Final index and term freq instead of obtaining them from running the program

In [38]:
# No need to run previous code. Just run this block to load Final index in (takes time to compute)

In [12]:
#Merged_block = open("Final_index")
Merged_block = pickle.load(open("final_index_concordia",mode='rb'))

In [13]:
#Term frequency
term_freq = pickle.load(open("term_freq_concordia",mode="rb"))

# Ranking Code

In [14]:
"Another approach to get length of a document.. Counts the number of times a token occured in some document(its length)."
def get_doc_length(doc_id):
    
    os.chdir("documents-final")
    f = open(str(doc_id)+".txt","r",encoding="utf=8")
    text= f.read()
    os.chdir("..")
    
    return len(text.split(" "))

"Calculates the average length of all documents using the get_doc_length method."
def get_collection_length():
    
    total = 0
    
    os.chdir("documents-final")
    docs = os.listdir()
    os.chdir("..")
    
    for doc in docs:
        if doc.split(".")[0].isdigit():
            total+=get_doc_length(doc)
    
    
        
    return total/40211
        

"BM25 Ranking formula implementation"
def rank_doc(doc_id,term):
        
    N = 17393 # Total documents number
    k1 = 0.25 # tuning parameter
    b = 0.75 # tuning parameter
    postings_list = []
    
    avg_doc_length = 200.04749944045162


    doc_length = get_doc_length(doc_id)
        
    
    "Find term frequency"
    
    term_frequency = term_freq[term]
    
    "Find document frequency"
    document_frequency = len(Merged_block[term])
    
    "Formula steps:"
    
    idf = np.log(N/document_frequency)
    numerator = (k1+1)*term_frequency
    denomenator = k1*((1-b)+ (b*(doc_length/avg_doc_length))) + term_frequency
    
    ratio = numerator/denomenator
    
    weight = idf * ratio
    
    return weight

def rank_doc_tfidf(doc_id,term):
    
    N = 17393 # Total documents number
    postings_list = []
    
    avg_doc_length = 200.04749944045162

    doc_length = get_doc_length(doc_id)
        
    
    "Find term frequency"
    
    term_frequency = term_freq[term]
    
    "Find document frequency"
    document_frequency = len(Merged_block[term])
            
    
     
    if document_frequency == 0:
        #"term doesn't exist in document"
        return 0
    
    "Formula steps:"
    
    idf = np.log(N/document_frequency)
    tf = 1 + np.log(term_frequency)
    weight = idf*tf
    
    
    return weight


In [15]:
#os.chdir("..")

In [16]:
#get_collection_length()

In [17]:
#os.getcwd()

# Querying code

In [18]:
"This function can run a single query, multi-term AND query, and a multi-term AND query with 1 OR."

def retrieve_docs(query,OR=False,vector_norm=False,tfidf=False,num_ret=10):
    
    terms = query.split(" ")
    temp_postings = []
    postings_list = []
    "Run the following if its a single term query"
    
    if len(terms) == 1:
            
        if terms[0] in Merged_block.keys():
            documents = set(Merged_block[terms[0]])
        else:
            print("Term doesn't exist in the IR system.")
            return
        
        "To store ranked documents"
        ranked_dict = {}
        
        "Rank documents and store document-rankingweight mappings"
        
        for document in documents:
            if tfidf == False:
                ranked_dict[document] = rank_doc(document,terms[0])
            else:
                ranked_dict[document] = rank_doc_tfidf(document,terms[0])


        
        "Sort ranked results"
        ranked_dict = sorted(ranked_dict.items(), key=itemgetter(1),reverse=True)
        
        return ranked_dict
        
    "Retrieve the postings of that term"
    for term in terms:
        if (term) in Merged_block.keys():
            postings_list = list(set(Merged_block[(term)]))
                
           
        if len(postings_list) == 0:
            print("Term not found in IR system, and thus will be ignored.")
            postings_list = []
            continue
        temp_postings.append(postings_list)    
        
        
    "Optimizes querying by dealing with the shortest postings lists first"    
    temp_postings.sort(key=len)
    
    documents = set()
    
    "AND + OR logic"
    for i in range(len(temp_postings)):
        if i == 0:
            documents = documents.union(set(temp_postings[i]))
        if i == 1 and OR == True:
            documents = documents.union(set(temp_postings[i]))
        else:
            documents = documents.intersection(set(temp_postings[i]))
     
    "Store document-rankingWeight mappings"
    ranked_dict = {}
    "Accumaltor for all the query terms in a doc as per the BM25 formula"
    total_rank = 0
    
    for document in documents:
        for term in terms:
            if tfidf == False:
                total_rank+= rank_doc(document,term)
            else:
                total_rank+= rank_doc_tfidf(document,term)


        ranked_dict[document] = total_rank
        
    "Sort ranked results"       
    ranked_dict = sorted(ranked_dict.items(), key=itemgetter(1),reverse=True)
    
    #return list(temp_dict.keys())[:10]
    return ranked_dict[:num_ret]



#  Which departments have AI research

In [19]:
query1 = "department artificial intelligence concordia research" # concordia is important cuz other school could be referenced
retrieve_docs((query1),tfidf=False,num_ret=10)

[(30841, 962.5606007696398),
 (26871, 949.8557620881151),
 (19951, 937.6610006131345),
 (24308, 924.973993656531),
 (13428, 912.2566733383641),
 (6514, 899.5412261015653),
 (12018, 887.3466477761266),
 (21744, 874.9471366234667),
 (14576, 862.2368429142776),
 (16239, 849.5207799523987)]

In [20]:
query2 = "department artificial intelligence research researcher engineering machine learning"
retrieve_docs((query2),tfidf=False,num_ret=10)

[(12478, 251.05167838678938),
 (26871, 225.65185976762305),
 (13781, 201.1616822888468),
 (26036, 175.7416580416431),
 (6514, 150.34236513415468),
 (6033, 125.85251807237526),
 (34637, 100.44675452371919),
 (21484, 75.32245975796543),
 (34918, 49.91578126184317),
 (13889, 25.425913549365845)]

In [21]:
query3 = "artificial intelligence ai research concordia faculty department university"
retrieve_docs((query3),tfidf=False,num_ret=10)

[(33565, 258.7621532100718),
 (11292, 238.69117445447435),
 (26871, 218.61039560194226),
 (24308, 199.23363627513785),
 (10259, 179.15232403933504),
 (6514, 159.1621234173823),
 (33326, 139.78561239771815),
 (10093, 119.70430016191533),
 (25004, 99.69530850394206),
 (1480, 79.61263681643665)]

# Which researchers are working on AI research?

In [22]:
query1 = "concordia computer science artificial intelligence engineering university researcher computing"
retrieve_docs((query1),tfidf=False,num_ret=10)

[(26871, 157.5556548564489),
 (949, 131.85745874763847),
 (31283, 105.033512234989),
 (6514, 78.20942435382204),
 (6033, 52.511625661462865),
 (34918, 25.69782353034346)]

In [23]:
query2 = "natural language processing researcher"
retrieve_docs((query2),tfidf=False,num_ret=10)

[(33790, 120.51758971570958),
 (26871, 105.28609946192421),
 (949, 90.52679888632926),
 (31283, 75.27287641477862),
 (6514, 60.01889337098953),
 (6033, 45.25977179364042),
 (23820, 30.010196888342495),
 (34918, 14.75913276458623)]

In [24]:
query3 = "researcher concordia deep learning phd engineering" 
retrieve_docs((query3),tfidf=False,num_ret=10)

[(10102, 130.2351807893794),
 (949, 113.91024731162511),
 (31283, 97.59175352668551),
 (6033, 81.2731973865194),
 (11151, 64.95917841377683),
 (2958, 48.920818483518225),
 (12652, 32.59378634009286),
 (5412, 16.312412135657354)]

# What AI research is being conducted at Concordia

In [25]:
query1 = "concordia university artificial intelligence research paper graduate conducted"
retrieve_docs((query1),tfidf=False,num_ret=10)

[(4511, 107.08645939474773),
 (26871, 85.35389222125066),
 (15923, 64.14914183600933),
 (6514, 42.40889800254895),
 (34918, 21.20445872314173)]

In [26]:
query2 = "concordia artificial intelligence research neural network deep learning"
retrieve_docs((query2),tfidf=False,num_ret=10)

[(18950, 107.58476591117675),
 (11460, 80.92900926271241),
 (10259, 53.62098267741364),
 (12018, 26.45185196966367)]

In [27]:
query3 = "ai research concordia"
retrieve_docs((query3),tfidf=False,num_ret=10)

[(25599, 682.7112824985373),
 (11249, 676.1435224441726),
 (29169, 669.571428662156),
 (8678, 663.0034400408281),
 (1480, 656.4368182686878),
 (14787, 649.8662758566022),
 (21435, 643.2943771098338),
 (13243, 636.7223935670512),
 (13239, 630.1634296930455),
 (33206, 623.5962578882013)]